<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-и-подготовка-данных-для-BERT" data-toc-modified-id="Изучение-и-подготовка-данных-для-BERT-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение и подготовка данных для BERT</a></span></li><li><span><a href="#Построение-моделей" data-toc-modified-id="Построение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Построение моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li><li><span><a href="#Модель-LightGBM" data-toc-modified-id="Модель-LightGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель LightGBM</a></span></li><li><span><a href="#Модель-CatBoost" data-toc-modified-id="Модель-CatBoost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель CatBoost</a></span></li></ul></li><li><span><a href="#Вывод-по-BERT:" data-toc-modified-id="Вывод-по-BERT:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод по BERT:</a></span></li><li><span><a href="#Подготовка-данных-для-классификации-текстов" data-toc-modified-id="Подготовка-данных-для-классификации-текстов-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Подготовка данных для классификации текстов</a></span></li><li><span><a href="#Построение-моделей" data-toc-modified-id="Построение-моделей-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Построение моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li><li><span><a href="#Модель-LigthGBM" data-toc-modified-id="Модель-LigthGBM-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Модель LigthGBM</a></span></li><li><span><a href="#Модель-CatBoost" data-toc-modified-id="Модель-CatBoost-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Модель CatBoost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Изучение и подготовка данных для BERT

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import GridSearchCV

import re
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Код для чтения данных
try:
    df_toxic = pd.read_csv('df_toxic.csv')
except:
    df_toxic= pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
    df_toxic.to_csv('df_toxic.csv', index=False)

In [3]:
# Будем работать только с 1000 строками ради сокращения времени выполнения кода
df_toxic = df_toxic.sample(n=1000, random_state=12345).reset_index(drop=True)

In [4]:
df_toxic

,text,toxic
0,Ahh shut the fuck up you douchebag sand nigger...,1
1,"""\n\nREPLY: There is no such thing as Texas Co...",0
2,"Reply\nHey, you could at least mention Jasenov...",0
3,"Thats fine, there is no deadline ) chi?",0
4,"""\n\nDYK nomination of Mustarabim\n Hello! You...",0
...,...,...
995,"Date warriors \n\nHi, Hertz. Nice catch on Sec...",0
996,REDIRECT Talk:2013 Men's World Ice Hockey Cham...,0
997,This article is one of the most well cited art...,0
998,Harry the point you are making is pointless. I...,0


In [5]:
df_toxic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   toxic   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [6]:
# Найдем дубликаты
df_toxic[df_toxic.duplicated() == True]

,text,toxic


In [7]:
# Найдем распределение оценок
df_toxic['toxic'].value_counts()

0    901
1     99
Name: toxic, dtype: int64

In [8]:
# Загрузка предобученной модели DistilBERT
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel,
                                                    ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# Загрузка предобученной модели/токенизатора 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# Токенизируем каждый комментарий
tokenized = df_toxic['text'].apply((lambda x:
                                    tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (862 > 512). Running this sequence through the model will result in indexing errors


In [10]:
# Найдем максимальную длину векторов после токенизации
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# Применим padding к векторам
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [11]:
# Создадим маску для выделения важных токенов
attention_mask = np.where(padded != 0, 1, 0)

In [12]:
attention_mask.shape

(1000, 4645)

In [13]:
# Ограничим количество токенов
padded_01 = padded[:,:512]
attention_mask_01 = attention_mask[:,:512]

In [14]:
# Сделаем цикл по батчам
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded_01.shape[0] // batch_size)):
        # Преобразуем данные в формат тензоров
        batch = torch.LongTensor(padded_01[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask_01[batch_size*i:batch_size*(i+1)])
        # Укажем, что градиенты не нужны: модель BERT обучать не будем.
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        # # преобразуем элементы методом numpy() к типу numpy.array
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

## Построение моделей

In [15]:
# Соберём все эмбеддинги в матрицу признаков
features = np.concatenate(embeddings)
target = df_toxic['toxic']
# Разделим данные на тренировочную и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

### Модель логистической регрессии

In [25]:
model_lr = LogisticRegression(random_state=12345)
result_lr = cross_val_score(model_lr, features_train, target_train,
                                 scoring='f1').mean()
print('F1-мера модели логистической регрессии:', result_lr)

F1-мера модели логистической регрессии: 0.7165217391304347


/opt/anaconda3/envs/practicum/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/envs/practicum/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

In [26]:
# Проверим F1-меру на тестовой выборке
model_lr.fit(features_train, target_train)
predictions_lr = model_lr.predict(features_test)
print('F1-мера тестовой выборки', f1_score(target_test, predictions_lr))

F1-мера тестовой выборки 0.7599999999999999


/opt/anaconda3/envs/practicum/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Модель LightGBM

In [18]:
model_lgbm = LGBMClassifier(random_state=12345)
params_lgbm = {'n_estimators' : [100, 200, 500],
               'max_depth' : range(2, 7)}

search_lgbm = GridSearchCV(estimator=model_lgbm, param_grid=params_lgbm, scoring='f1',
                           verbose=3, cv=5)
result_lgbm = search_lgbm.fit(features_train , target_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END .....max_depth=2, n_estimators=100;, score=0.571 total time=   0.2s
[CV 2/5] END .....max_depth=2, n_estimators=100;, score=0.783 total time=   0.1s
[CV 3/5] END .....max_depth=2, n_estimators=100;, score=0.500 total time=   0.1s
[CV 4/5] END .....max_depth=2, n_estimators=100;, score=0.667 total time=   0.1s
[CV 5/5] END .....max_depth=2, n_estimators=100;, score=0.692 total time=   0.1s
[CV 1/5] END .....max_depth=2, n_estimators=200;, score=0.636 total time=   0.2s
[CV 2/5] END .....max_depth=2, n_estimators=200;, score=0.727 total time=   0.2s
[CV 3/5] END .....max_depth=2, n_estimators=200;, score=0.500 total time=   0.2s
[CV 4/5] END .....max_depth=2, n_estimators=200;, score=0.600 total time=   0.2s
[CV 5/5] END .....max_depth=2, n_estimators=200;, score=0.692 total time=   0.2s
[CV 1/5] END .....max_depth=2, n_estimators=500;, score=0.636 total time=   0.5s
[CV 2/5] END .....max_depth=2, n_estimators=500;

[CV 2/5] END .....max_depth=6, n_estimators=200;, score=0.667 total time=   0.8s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[CV 3/5] END .....max_depth=6, n_estimators=200;, score=0.476 total time=   0.8s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[CV 4/5] END .....max_depth=6, n_estimators=200;, score=0.571 total time=   0.8s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[CV 5/5] END .....max_depth=6, n_estimators=200;, score=0.720 total time=   0.8s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[CV 1/5] END .....max_depth=6, n_estimators=500;, score=0.600 total time=   0.9s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set nu

In [19]:
print('Лучшее значение F1-меры в LightGBM:', result_lgbm.best_score_)
print('Лучшие параметры LightGBM:', result_lgbm.best_params_)

Лучшее значение F1-меры в LightGBM: 0.6426023252110209
Лучшие параметры LightGBM: {'max_depth': 2, 'n_estimators': 100}


In [20]:
# Проверим модель LightGBM на тестовой выборке
best_model_lgbm = search_lgbm.best_estimator_
best_model_lgbm.fit(features_train, target_train)
predictions_lgbm=best_model_lgbm.predict(features_test)
print("F1-мера лучшей модели на тестовой выборке:", f1_score(target_test, predictions_lgbm))

F1-мера лучшей модели на тестовой выборке: 0.6666666666666667


### Модель CatBoost

In [21]:
model_cat = CatBoostClassifier(random_seed=12345)
params_cat = {'iterations' : [100, 300, 500],
              'depth' : range(4, 8)}

search_cat = GridSearchCV(estimator=model_cat, param_grid=params_cat, scoring='f1',
                           verbose=1, cv=3)
result_cat = search_cat.fit(features_train, target_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Learning rate set to 0.063299
0:	learn: 0.6195273	total: 79.9ms	remaining: 7.91s
1:	learn: 0.5661801	total: 90.7ms	remaining: 4.45s
2:	learn: 0.5104177	total: 102ms	remaining: 3.29s
3:	learn: 0.4674025	total: 114ms	remaining: 2.73s
4:	learn: 0.4229842	total: 126ms	remaining: 2.4s
5:	learn: 0.3874266	total: 138ms	remaining: 2.17s
6:	learn: 0.3529163	total: 150ms	remaining: 1.99s
7:	learn: 0.3255369	total: 161ms	remaining: 1.85s
8:	learn: 0.3033351	total: 172ms	remaining: 1.74s
9:	learn: 0.2848788	total: 183ms	remaining: 1.64s
10:	learn: 0.2668043	total: 193ms	remaining: 1.56s
11:	learn: 0.2553160	total: 205ms	remaining: 1.5s
12:	learn: 0.2395211	total: 218ms	remaining: 1.46s
13:	learn: 0.2267390	total: 229ms	remaining: 1.4s
14:	learn: 0.2138451	total: 240ms	remaining: 1.36s
15:	learn: 0.2062809	total: 251ms	remaining: 1.32s
16:	learn: 0.1956071	total: 262ms	remaining: 1.28s
17:	learn: 0.1848688	total: 273ms	remaining: 1.24s
18

65:	learn: 0.0575123	total: 746ms	remaining: 384ms
66:	learn: 0.0561033	total: 758ms	remaining: 373ms
67:	learn: 0.0550902	total: 769ms	remaining: 362ms
68:	learn: 0.0549487	total: 780ms	remaining: 351ms
69:	learn: 0.0533496	total: 791ms	remaining: 339ms
70:	learn: 0.0531841	total: 803ms	remaining: 328ms
71:	learn: 0.0528080	total: 814ms	remaining: 317ms
72:	learn: 0.0514311	total: 825ms	remaining: 305ms
73:	learn: 0.0507822	total: 836ms	remaining: 294ms
74:	learn: 0.0492526	total: 848ms	remaining: 283ms
75:	learn: 0.0482117	total: 860ms	remaining: 271ms
76:	learn: 0.0480418	total: 871ms	remaining: 260ms
77:	learn: 0.0470271	total: 882ms	remaining: 249ms
78:	learn: 0.0468862	total: 895ms	remaining: 238ms
79:	learn: 0.0453431	total: 906ms	remaining: 226ms
80:	learn: 0.0448390	total: 917ms	remaining: 215ms
81:	learn: 0.0432467	total: 928ms	remaining: 204ms
82:	learn: 0.0429340	total: 939ms	remaining: 192ms
83:	learn: 0.0424392	total: 951ms	remaining: 181ms
84:	learn: 0.0414263	total: 964

29:	learn: 0.2720812	total: 341ms	remaining: 3.07s
30:	learn: 0.2648274	total: 353ms	remaining: 3.07s
31:	learn: 0.2591533	total: 365ms	remaining: 3.05s
32:	learn: 0.2538454	total: 376ms	remaining: 3.04s
33:	learn: 0.2475272	total: 387ms	remaining: 3.03s
34:	learn: 0.2414170	total: 398ms	remaining: 3.02s
35:	learn: 0.2351534	total: 409ms	remaining: 3s
36:	learn: 0.2307314	total: 420ms	remaining: 2.99s
37:	learn: 0.2261149	total: 432ms	remaining: 2.98s
38:	learn: 0.2212603	total: 443ms	remaining: 2.96s
39:	learn: 0.2162433	total: 455ms	remaining: 2.96s
40:	learn: 0.2122771	total: 467ms	remaining: 2.95s
41:	learn: 0.2088018	total: 478ms	remaining: 2.94s
42:	learn: 0.2045373	total: 490ms	remaining: 2.93s
43:	learn: 0.2006403	total: 501ms	remaining: 2.91s
44:	learn: 0.1973888	total: 512ms	remaining: 2.9s
45:	learn: 0.1931488	total: 524ms	remaining: 2.89s
46:	learn: 0.1888881	total: 535ms	remaining: 2.88s
47:	learn: 0.1846547	total: 546ms	remaining: 2.87s
48:	learn: 0.1812279	total: 558ms	r

191:	learn: 0.0495564	total: 2.2s	remaining: 1.24s
192:	learn: 0.0491687	total: 2.21s	remaining: 1.23s
193:	learn: 0.0485676	total: 2.22s	remaining: 1.22s
194:	learn: 0.0481899	total: 2.24s	remaining: 1.2s
195:	learn: 0.0478528	total: 2.25s	remaining: 1.19s
196:	learn: 0.0474142	total: 2.26s	remaining: 1.18s
197:	learn: 0.0473494	total: 2.27s	remaining: 1.17s
198:	learn: 0.0470698	total: 2.28s	remaining: 1.16s
199:	learn: 0.0465086	total: 2.29s	remaining: 1.15s
200:	learn: 0.0460585	total: 2.3s	remaining: 1.14s
201:	learn: 0.0459628	total: 2.32s	remaining: 1.12s
202:	learn: 0.0459007	total: 2.33s	remaining: 1.11s
203:	learn: 0.0457771	total: 2.34s	remaining: 1.1s
204:	learn: 0.0455288	total: 2.35s	remaining: 1.09s
205:	learn: 0.0451943	total: 2.36s	remaining: 1.08s
206:	learn: 0.0447410	total: 2.37s	remaining: 1.07s
207:	learn: 0.0446874	total: 2.38s	remaining: 1.05s
208:	learn: 0.0446413	total: 2.4s	remaining: 1.04s
209:	learn: 0.0445051	total: 2.41s	remaining: 1.03s
210:	learn: 0.043

59:	learn: 0.1487383	total: 696ms	remaining: 2.78s
60:	learn: 0.1464228	total: 709ms	remaining: 2.78s
61:	learn: 0.1438930	total: 720ms	remaining: 2.77s
62:	learn: 0.1420744	total: 732ms	remaining: 2.75s
63:	learn: 0.1407646	total: 743ms	remaining: 2.74s
64:	learn: 0.1392600	total: 754ms	remaining: 2.73s
65:	learn: 0.1375005	total: 766ms	remaining: 2.71s
66:	learn: 0.1355190	total: 777ms	remaining: 2.7s
67:	learn: 0.1335862	total: 789ms	remaining: 2.69s
68:	learn: 0.1318109	total: 801ms	remaining: 2.68s
69:	learn: 0.1297884	total: 812ms	remaining: 2.67s
70:	learn: 0.1275858	total: 824ms	remaining: 2.66s
71:	learn: 0.1268696	total: 836ms	remaining: 2.65s
72:	learn: 0.1250676	total: 847ms	remaining: 2.63s
73:	learn: 0.1239534	total: 859ms	remaining: 2.62s
74:	learn: 0.1222091	total: 870ms	remaining: 2.61s
75:	learn: 0.1204798	total: 881ms	remaining: 2.6s
76:	learn: 0.1188045	total: 893ms	remaining: 2.59s
77:	learn: 0.1173305	total: 905ms	remaining: 2.58s
78:	learn: 0.1157003	total: 917ms

...

272:	learn: 0.0350920	total: 15.9s	remaining: 13.2s
273:	learn: 0.0349335	total: 15.9s	remaining: 13.1s
274:	learn: 0.0347338	total: 16s	remaining: 13.1s
275:	learn: 0.0346150	total: 16s	remaining: 13s
276:	learn: 0.0343895	total: 16.1s	remaining: 13s
277:	learn: 0.0341488	total: 16.2s	remaining: 12.9s
278:	learn: 0.0339155	total: 16.2s	remaining: 12.9s
279:	learn: 0.0337038	total: 16.3s	remaining: 12.8s
280:	learn: 0.0335257	total: 16.3s	remaining: 12.7s
281:	learn: 0.0333321	total: 16.4s	remaining: 12.7s
282:	learn: 0.0331547	total: 16.5s	remaining: 12.6s
283:	learn: 0.0330574	total: 16.5s	remaining: 12.6s
284:	learn: 0.0329341	total: 16.6s	remaining: 12.5s
285:	learn: 0.0327351	total: 16.6s	remaining: 12.4s
286:	learn: 0.0325731	total: 16.7s	remaining: 12.4s
287:	learn: 0.0323631	total: 16.7s	remaining: 12.3s
288:	learn: 0.0322125	total: 16.8s	remaining: 12.3s
289:	learn: 0.0321042	total: 16.9s	remaining: 12.2s
290:	learn: 0.0319751	total: 16.9s	remaining: 12.1s
291:	learn: 0.031762

432:	learn: 0.0170098	total: 25.1s	remaining: 3.88s
433:	learn: 0.0169289	total: 25.1s	remaining: 3.82s
434:	learn: 0.0168575	total: 25.2s	remaining: 3.76s
435:	learn: 0.0167623	total: 25.2s	remaining: 3.7s
436:	learn: 0.0167134	total: 25.3s	remaining: 3.64s
437:	learn: 0.0166437	total: 25.3s	remaining: 3.59s
438:	learn: 0.0165739	total: 25.4s	remaining: 3.53s
439:	learn: 0.0165381	total: 25.4s	remaining: 3.47s
440:	learn: 0.0164443	total: 25.5s	remaining: 3.41s
441:	learn: 0.0163649	total: 25.6s	remaining: 3.35s
442:	learn: 0.0162826	total: 25.6s	remaining: 3.3s
443:	learn: 0.0162222	total: 25.7s	remaining: 3.24s
444:	learn: 0.0161652	total: 25.8s	remaining: 3.18s
445:	learn: 0.0160916	total: 25.8s	remaining: 3.13s
446:	learn: 0.0160180	total: 25.9s	remaining: 3.07s
447:	learn: 0.0159647	total: 25.9s	remaining: 3.01s
448:	learn: 0.0158945	total: 26s	remaining: 2.95s
449:	learn: 0.0158028	total: 26.1s	remaining: 2.89s
450:	learn: 0.0157379	total: 26.1s	remaining: 2.84s
451:	learn: 0.01

103:	learn: 0.1360094	total: 1.37s	remaining: 5.21s
104:	learn: 0.1349715	total: 1.39s	remaining: 5.21s
105:	learn: 0.1339988	total: 1.4s	remaining: 5.2s
106:	learn: 0.1333332	total: 1.41s	remaining: 5.19s
107:	learn: 0.1323065	total: 1.42s	remaining: 5.17s
108:	learn: 0.1314626	total: 1.44s	remaining: 5.16s
109:	learn: 0.1307703	total: 1.45s	remaining: 5.14s
110:	learn: 0.1299123	total: 1.46s	remaining: 5.13s
111:	learn: 0.1290791	total: 1.48s	remaining: 5.12s
112:	learn: 0.1281853	total: 1.49s	remaining: 5.1s
113:	learn: 0.1271441	total: 1.5s	remaining: 5.09s
114:	learn: 0.1260938	total: 1.51s	remaining: 5.07s
115:	learn: 0.1257861	total: 1.53s	remaining: 5.06s
116:	learn: 0.1253879	total: 1.54s	remaining: 5.04s
117:	learn: 0.1242468	total: 1.55s	remaining: 5.03s
118:	learn: 0.1237048	total: 1.57s	remaining: 5.02s
119:	learn: 0.1228592	total: 1.58s	remaining: 5s
120:	learn: 0.1221057	total: 1.59s	remaining: 5s
121:	learn: 0.1216225	total: 1.61s	remaining: 4.98s
122:	learn: 0.1210417	

263:	learn: 0.0680517	total: 3.44s	remaining: 3.08s
264:	learn: 0.0676277	total: 3.46s	remaining: 3.06s
265:	learn: 0.0671365	total: 3.47s	remaining: 3.05s
266:	learn: 0.0670645	total: 3.48s	remaining: 3.04s
267:	learn: 0.0667662	total: 3.5s	remaining: 3.03s
268:	learn: 0.0666453	total: 3.51s	remaining: 3.01s
269:	learn: 0.0663948	total: 3.52s	remaining: 3s
270:	learn: 0.0660256	total: 3.53s	remaining: 2.99s
271:	learn: 0.0658642	total: 3.55s	remaining: 2.97s
272:	learn: 0.0656346	total: 3.56s	remaining: 2.96s
273:	learn: 0.0653682	total: 3.57s	remaining: 2.95s
274:	learn: 0.0651303	total: 3.58s	remaining: 2.93s
275:	learn: 0.0646332	total: 3.6s	remaining: 2.92s
276:	learn: 0.0643165	total: 3.61s	remaining: 2.91s
277:	learn: 0.0641442	total: 3.62s	remaining: 2.89s
278:	learn: 0.0640111	total: 3.64s	remaining: 2.88s
279:	learn: 0.0635618	total: 3.65s	remaining: 2.87s
280:	learn: 0.0631927	total: 3.66s	remaining: 2.85s
281:	learn: 0.0628880	total: 3.68s	remaining: 2.84s
282:	learn: 0.062

423:	learn: 0.0385391	total: 5.51s	remaining: 988ms
424:	learn: 0.0383888	total: 5.53s	remaining: 975ms
425:	learn: 0.0382476	total: 5.54s	remaining: 962ms
426:	learn: 0.0382289	total: 5.55s	remaining: 949ms
427:	learn: 0.0381797	total: 5.56s	remaining: 936ms
428:	learn: 0.0381043	total: 5.58s	remaining: 923ms
429:	learn: 0.0378708	total: 5.59s	remaining: 910ms
430:	learn: 0.0377426	total: 5.6s	remaining: 897ms
431:	learn: 0.0375882	total: 5.62s	remaining: 884ms
432:	learn: 0.0373075	total: 5.63s	remaining: 871ms
433:	learn: 0.0372680	total: 5.64s	remaining: 858ms
434:	learn: 0.0372534	total: 5.66s	remaining: 845ms
435:	learn: 0.0370838	total: 5.67s	remaining: 832ms
436:	learn: 0.0369591	total: 5.68s	remaining: 819ms
437:	learn: 0.0367631	total: 5.69s	remaining: 806ms
438:	learn: 0.0366515	total: 5.71s	remaining: 793ms
439:	learn: 0.0366265	total: 5.72s	remaining: 780ms
440:	learn: 0.0364158	total: 5.74s	remaining: 767ms
441:	learn: 0.0363206	total: 5.75s	remaining: 754ms
442:	learn: 0

In [22]:
print('Лучшее значение F1-меры в CatBoost:', result_cat.best_score_)
print('Лучшие параметры CatBoost:', result_cat.best_params_)

Лучшее значение F1-меры в CatBoost: 0.4985682597297842
Лучшие параметры CatBoost: {'depth': 4, 'iterations': 500}


In [23]:
# Проверим модель LightGBM на тестовой выборке
best_model_cat = search_cat.best_estimator_
best_model_cat.fit(features_train, target_train)
predictions_cat=best_model_cat.predict(features_test)

Learning rate set to 0.017204
0:	learn: 0.6747612	total: 12.8ms	remaining: 6.36s
1:	learn: 0.6586915	total: 24.9ms	remaining: 6.21s
2:	learn: 0.6394685	total: 37.2ms	remaining: 6.16s
3:	learn: 0.6229377	total: 49.7ms	remaining: 6.16s
4:	learn: 0.6021268	total: 63ms	remaining: 6.23s
5:	learn: 0.5857017	total: 76.1ms	remaining: 6.27s
6:	learn: 0.5700984	total: 88.7ms	remaining: 6.25s
7:	learn: 0.5534114	total: 101ms	remaining: 6.24s
8:	learn: 0.5413213	total: 114ms	remaining: 6.24s
9:	learn: 0.5283144	total: 127ms	remaining: 6.21s
10:	learn: 0.5133013	total: 139ms	remaining: 6.2s
11:	learn: 0.5036730	total: 152ms	remaining: 6.18s
12:	learn: 0.4929720	total: 165ms	remaining: 6.18s
13:	learn: 0.4801533	total: 180ms	remaining: 6.23s
14:	learn: 0.4666687	total: 192ms	remaining: 6.21s
15:	learn: 0.4563282	total: 208ms	remaining: 6.28s
16:	learn: 0.4438585	total: 221ms	remaining: 6.27s
17:	learn: 0.4310014	total: 234ms	remaining: 6.28s
18:	learn: 0.4215554	total: 247ms	remaining: 6.25s
19:	lea

171:	learn: 0.0965922	total: 2.25s	remaining: 4.29s
172:	learn: 0.0961181	total: 2.26s	remaining: 4.28s
173:	learn: 0.0955388	total: 2.28s	remaining: 4.27s
174:	learn: 0.0948129	total: 2.29s	remaining: 4.25s
175:	learn: 0.0944628	total: 2.3s	remaining: 4.24s
176:	learn: 0.0942092	total: 2.32s	remaining: 4.23s
177:	learn: 0.0939037	total: 2.33s	remaining: 4.21s
178:	learn: 0.0932346	total: 2.34s	remaining: 4.2s
179:	learn: 0.0926536	total: 2.35s	remaining: 4.18s
180:	learn: 0.0922282	total: 2.37s	remaining: 4.17s
181:	learn: 0.0918499	total: 2.38s	remaining: 4.16s
182:	learn: 0.0916631	total: 2.39s	remaining: 4.14s
183:	learn: 0.0915019	total: 2.4s	remaining: 4.13s
184:	learn: 0.0914224	total: 2.42s	remaining: 4.12s
185:	learn: 0.0911457	total: 2.43s	remaining: 4.1s
186:	learn: 0.0906068	total: 2.44s	remaining: 4.09s
187:	learn: 0.0899350	total: 2.46s	remaining: 4.08s
188:	learn: 0.0893878	total: 2.47s	remaining: 4.07s
189:	learn: 0.0888940	total: 2.48s	remaining: 4.05s
190:	learn: 0.08

331:	learn: 0.0531668	total: 4.33s	remaining: 2.19s
332:	learn: 0.0528212	total: 4.34s	remaining: 2.18s
333:	learn: 0.0526526	total: 4.35s	remaining: 2.16s
334:	learn: 0.0523236	total: 4.37s	remaining: 2.15s
335:	learn: 0.0519303	total: 4.38s	remaining: 2.14s
336:	learn: 0.0516968	total: 4.39s	remaining: 2.13s
337:	learn: 0.0515214	total: 4.41s	remaining: 2.11s
338:	learn: 0.0513378	total: 4.42s	remaining: 2.1s
339:	learn: 0.0512435	total: 4.43s	remaining: 2.09s
340:	learn: 0.0509456	total: 4.45s	remaining: 2.07s
341:	learn: 0.0508716	total: 4.46s	remaining: 2.06s
342:	learn: 0.0507020	total: 4.47s	remaining: 2.05s
343:	learn: 0.0504861	total: 4.49s	remaining: 2.03s
344:	learn: 0.0503233	total: 4.5s	remaining: 2.02s
345:	learn: 0.0502639	total: 4.51s	remaining: 2.01s
346:	learn: 0.0499944	total: 4.52s	remaining: 1.99s
347:	learn: 0.0499729	total: 4.54s	remaining: 1.98s
348:	learn: 0.0497724	total: 4.55s	remaining: 1.97s
349:	learn: 0.0495493	total: 4.56s	remaining: 1.96s
350:	learn: 0.

491:	learn: 0.0307759	total: 6.41s	remaining: 104ms
492:	learn: 0.0306122	total: 6.42s	remaining: 91.2ms
493:	learn: 0.0305947	total: 6.44s	remaining: 78.2ms
494:	learn: 0.0304124	total: 6.45s	remaining: 65.2ms
495:	learn: 0.0302942	total: 6.46s	remaining: 52.1ms
496:	learn: 0.0302515	total: 6.48s	remaining: 39.1ms
497:	learn: 0.0302373	total: 6.49s	remaining: 26.1ms
498:	learn: 0.0302203	total: 6.5s	remaining: 13ms
499:	learn: 0.0300245	total: 6.52s	remaining: 0us


In [24]:
print("F1-мера лучшей модели на тестовой выборке:", f1_score(target_test, predictions_cat))

F1-мера лучшей модели на тестовой выборке: 0.5714285714285715


## Вывод по BERT: 
Работает долго, поэтому построил модели только на малой части данных (1000 случайных строк). Удивительно, но на логистической регресси каким то образом получилась f1-мера > 0.75=)

# Модель без участия BERT

## Подготовка данных для классификации текстов

In [2]:
# Загрузим данные
df_wikishop = pd.read_csv('df_toxic.csv')
corpus = list(df_wikishop['text'].values.astype('U'))

In [3]:
# Напишем функцию для лемматизации текста
nltk.download('wordnet')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()
def lemm(text):   
    word_list = nltk.word_tokenize(text)
    return' '.join([lemmatizer.lemmatize(w) for w in word_list])

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aleksandrverlan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aleksandrverlan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Напишем функцию для очистки текста
def clear_text(text):
    return ' '.join(re.sub(r'[^a-zA-Z]', ' ', text).split())

In [5]:
# Применим функции ко всем строкам
df_wikishop['text_lemm'] = df_wikishop['text'].apply(lambda x: lemm(clear_text(x)))

In [6]:
df_wikishop.head()

,text,toxic,text_lemm
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [7]:
df_wikishop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159571 non-null  object
 1   toxic      159571 non-null  int64 
 2   text_lemm  159571 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.7+ MB


In [8]:
# Разделим данные на тренировочную и тестовую выборки
train, test = train_test_split(df_wikishop, test_size=0.25, random_state=12345)

In [12]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
# Выделим переменные признаки и признак, который нужно предсказать
train_target=train['toxic']
train_corpus=train['text_lemm']
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
train_features = count_tf_idf.fit_transform(train_corpus) 
test_target=test['toxic']
test_corpus=test['text_lemm']
test_features = count_tf_idf.transform(test_corpus) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandrverlan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Построение моделей

In [13]:
# Создадим функцию обучения модели
def fit_predict(model):
    model.fit(train_features, train_target)
    predict=model.predict(test_features)
    f1=f1_score(test_target, predict)
    print("F1 модели на тестовой выборке:", f1)

### Модель логистической регрессии

In [14]:
print('Логистическая регрессия')
model_lr = LogisticRegression(random_state=12345)
fit_predict(model_lr)

Логистическая регрессия
F1 модели на тестовой выборке: 0.7374153664998528


### Модель LigthGBM

In [18]:
print('LightGBM')
model_lgb=LGBMClassifier(random_state=12345, n_estimators=500)
fit_predict(model_lgb)

LightGBM
F1 модели на тестовой выборке: 0.774986346258875


### Модель CatBoost

In [19]:
print('CatBoost')
model_cat=CatBoostClassifier(random_seed=12345, iterations=500)
fit_predict(model_cat)

CatBoost
Learning rate set to 0.150069
0:	learn: 0.5467665	total: 977ms	remaining: 8m 7s
1:	learn: 0.4518392	total: 1.71s	remaining: 7m 6s
2:	learn: 0.3855012	total: 2.18s	remaining: 6m 1s
3:	learn: 0.3389108	total: 2.65s	remaining: 5m 28s
4:	learn: 0.3079332	total: 3.12s	remaining: 5m 8s
5:	learn: 0.2842842	total: 3.64s	remaining: 4m 59s
6:	learn: 0.2681348	total: 4.11s	remaining: 4m 49s
7:	learn: 0.2569842	total: 4.58s	remaining: 4m 41s
8:	learn: 0.2478066	total: 5.07s	remaining: 4m 36s
9:	learn: 0.2410701	total: 5.57s	remaining: 4m 33s
10:	learn: 0.2357826	total: 6.05s	remaining: 4m 28s
11:	learn: 0.2302520	total: 6.55s	remaining: 4m 26s
12:	learn: 0.2265553	total: 7.04s	remaining: 4m 23s
13:	learn: 0.2237080	total: 7.49s	remaining: 4m 20s
14:	learn: 0.2209691	total: 7.95s	remaining: 4m 17s
15:	learn: 0.2176854	total: 8.41s	remaining: 4m 14s
16:	learn: 0.2153899	total: 8.87s	remaining: 4m 11s
17:	learn: 0.2135348	total: 9.33s	remaining: 4m 9s
18:	learn: 0.2116383	total: 9.78s	remain

157:	learn: 0.1409773	total: 1m 16s	remaining: 2m 46s
158:	learn: 0.1408610	total: 1m 17s	remaining: 2m 45s
159:	learn: 0.1406999	total: 1m 17s	remaining: 2m 45s
160:	learn: 0.1405787	total: 1m 18s	remaining: 2m 44s
161:	learn: 0.1403404	total: 1m 18s	remaining: 2m 43s
162:	learn: 0.1401237	total: 1m 19s	remaining: 2m 43s
163:	learn: 0.1399576	total: 1m 19s	remaining: 2m 42s
164:	learn: 0.1398417	total: 1m 20s	remaining: 2m 42s
165:	learn: 0.1396628	total: 1m 20s	remaining: 2m 41s
166:	learn: 0.1394324	total: 1m 20s	remaining: 2m 41s
167:	learn: 0.1392712	total: 1m 21s	remaining: 2m 40s
168:	learn: 0.1390109	total: 1m 21s	remaining: 2m 40s
169:	learn: 0.1388423	total: 1m 22s	remaining: 2m 39s
170:	learn: 0.1384974	total: 1m 22s	remaining: 2m 39s
171:	learn: 0.1383837	total: 1m 23s	remaining: 2m 38s
172:	learn: 0.1382817	total: 1m 23s	remaining: 2m 38s
173:	learn: 0.1381564	total: 1m 24s	remaining: 2m 37s
174:	learn: 0.1379561	total: 1m 24s	remaining: 2m 37s
175:	learn: 0.1377852	total:

310:	learn: 0.1212715	total: 2m 30s	remaining: 1m 31s
311:	learn: 0.1211478	total: 2m 30s	remaining: 1m 30s
312:	learn: 0.1210885	total: 2m 31s	remaining: 1m 30s
313:	learn: 0.1210435	total: 2m 31s	remaining: 1m 29s
314:	learn: 0.1209982	total: 2m 32s	remaining: 1m 29s
315:	learn: 0.1209485	total: 2m 32s	remaining: 1m 28s
316:	learn: 0.1208720	total: 2m 33s	remaining: 1m 28s
317:	learn: 0.1207864	total: 2m 33s	remaining: 1m 27s
318:	learn: 0.1206982	total: 2m 34s	remaining: 1m 27s
319:	learn: 0.1205861	total: 2m 34s	remaining: 1m 26s
320:	learn: 0.1205422	total: 2m 34s	remaining: 1m 26s
321:	learn: 0.1204988	total: 2m 35s	remaining: 1m 25s
322:	learn: 0.1204556	total: 2m 35s	remaining: 1m 25s
323:	learn: 0.1204123	total: 2m 36s	remaining: 1m 24s
324:	learn: 0.1203160	total: 2m 36s	remaining: 1m 24s
325:	learn: 0.1202127	total: 2m 37s	remaining: 1m 23s
326:	learn: 0.1201554	total: 2m 37s	remaining: 1m 23s
327:	learn: 0.1201111	total: 2m 38s	remaining: 1m 22s
328:	learn: 0.1199751	total:

465:	learn: 0.1103269	total: 3m 44s	remaining: 16.4s
466:	learn: 0.1102896	total: 3m 45s	remaining: 15.9s
467:	learn: 0.1101379	total: 3m 45s	remaining: 15.4s
468:	learn: 0.1100638	total: 3m 46s	remaining: 15s
469:	learn: 0.1100344	total: 3m 46s	remaining: 14.5s
470:	learn: 0.1099230	total: 3m 47s	remaining: 14s
471:	learn: 0.1098259	total: 3m 47s	remaining: 13.5s
472:	learn: 0.1097822	total: 3m 48s	remaining: 13s
473:	learn: 0.1096510	total: 3m 48s	remaining: 12.5s
474:	learn: 0.1096233	total: 3m 49s	remaining: 12.1s
475:	learn: 0.1095307	total: 3m 49s	remaining: 11.6s
476:	learn: 0.1094643	total: 3m 50s	remaining: 11.1s
477:	learn: 0.1093613	total: 3m 50s	remaining: 10.6s
478:	learn: 0.1093300	total: 3m 51s	remaining: 10.1s
479:	learn: 0.1092788	total: 3m 51s	remaining: 9.66s
480:	learn: 0.1092082	total: 3m 52s	remaining: 9.18s
481:	learn: 0.1091784	total: 3m 52s	remaining: 8.7s
482:	learn: 0.1090883	total: 3m 53s	remaining: 8.21s
483:	learn: 0.1090196	total: 3m 53s	remaining: 7.73s


Модели LightGBM и CatBoost выдали F1-меру > 0.75, но на наших данных LightGBM выиграла=) Логистическая регрессия была близка к ним!

## Выводы
- В ходе работы мы изучили данные
- Подготовили малую часть данных (1000 строк) с помощью предобученной модели DistilBERT
- Обучили различные модели: логистическую регрессию, LightGBM, CatBoost, и сравнили их F1-меру
- Подготовили данные с помощью TF-IDF и обучили те же модели на новых данных
- Получили F1-меру = 0.77 в модели LightGBM.